In [ ]:
import requests,re

class ICC_Ranking():
    def __init__(self,url):
        print("Here intial Main Method")
        self.load(url)
    
    def Insert_excel(self,mydict):
        print("Here")
        
        import xlwt
        from xlwt import Workbook
        import pandas as pd
        import openpyxl
                
        #write the data
        df = pd.DataFrame(mydict, columns=['Rank', 'Name', 'Team', 'Rating','Best'])
        df.to_excel('Icc_rankings.xlsx', index=False, sheet_name='Rank_sheet')
        
        print("succesfully write in excel")
        
        #read the data        
        filename='Icc_rankings.xlsx'
        
        book = openpyxl.load_workbook(filename)
        # Define variable to read sheet
        dataframe1 = book.active
         
        # Iterate the loop to read the cell values
        for row in range(0, dataframe1.max_row):
            for col in dataframe1.iter_cols(1, dataframe1.max_column):
                print(col[row].value)
        
    def load(self,url):
    
        headers = {
            'authority': 'www.icc-cricket.com',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
            # 'cookie': 'test; G_ENABLED_IDPS=google; _fbp=fb.1.1678197564347.1926737956; _parsely_session={%22sid%22:2%2C%22surl%22:%22https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting%22%2C%22sref%22:%22%22%2C%22sts%22:1678286818332%2C%22slts%22:1678197563669}; _parsely_visitor={%22id%22:%22pid=856765e1-2d07-4933-9100-818175162be2%22%2C%22session_count%22:2%2C%22last_session_ts%22:1678286818332}; _gid=GA1.2.199009834.1678286819; _ga=GA1.1.700286414.1678197564; _hjSessionUser_2924928=eyJpZCI6ImU2ODY1MDRkLWUzOTItNWExOC04YWZjLWMzMWNiNDIzNmYyMyIsImNyZWF0ZWQiOjE2NzgxOTc1NjQ0OTUsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_2924928=eyJpZCI6ImU0MzlkODJlLTc1YjQtNDk1OS1hNmRhLWIxMjZiZjMyNjkwYSIsImNyZWF0ZWQiOjE2NzgyODY4MTkyMDIsImluU2FtcGxlIjpmYWxzZX0=; _hjAbsoluteSessionInProgress=0; _ga_SB8SBQVNVM=GS1.1.1678286818.3.0.1678286958.60.0.0',
            'sec-ch-ua': '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Linux"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
        }
        
        response = requests.get('https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting', headers=headers)
        print("response :",response.status_code)
        html=response.text
        
    def extraction(self,html):
        
        mydict=[]
        if re.compile(r'(?s)<span class="rankings-.*?__pos-number">.*?</td>\s*</tr>').findall(html):
            for block in re.compile(r'(?s)<span class="rankings-.*?__pos-number">.*?</td>\s*</tr>').findall(html):
                # print("block :",block)
                
                Rank_reg = re.search(r'(?s)<span class="rankings-.*?__pos-number">(.*?)</span>', block)
                if Rank_reg:
                    Rank = Rank_reg.group(1).strip()
                    Rank = re.sub(r"<.*?>|=","",Rank)
                else:
                    Rank = ""
                
                Name_reg = re.search(r'<a href="/rankings/mens/player-rankings/.*?">(.*?)</a>', block)
                if Name_reg:
                    Name = Name_reg.group(1).strip()
                    Name = re.sub(r"<.*?>","",Name)
                else:
                    Name = ""
                    
                Team_reg = re.search(r'(?s)<span class="rankings-block__banner--nation"></span>(.*?)<', block)
                Team_reg_1 = re.search(r'<span class="table-body__logo-text">(.*?)</span>', block)
                if Team_reg:
                    Team = Team_reg.group(1).strip()
                    Team = re.sub(r"<.*?>","",Team)
                elif Team_reg_1:
                    Team = Team_reg_1.group(1).strip()
                    Team = re.sub(r"<.*?>","",Team)
                else:
                    Team = ""
                
                Rating_reg = re.search(r'banner--rating">(.*?)</div>', block)
                Rating_reg_1 = re.search(r'<td class="table-body__cell rating">(.*?)</td>', block)
                if Rating_reg:
                    Rating = Rating_reg.group(1).strip()
                elif Rating_reg_1:
                    Rating = Rating_reg_1.group(1).strip()
                else:
                    Rating = ""
                
                carrier_best_rating_reg = re.search(r'(?s)career-best-text">(.*?)</span>', block)
                carrier_best_rating_reg_1 = re.search(r'(?s)<td class="table-body__cell u-text-right u-hide-phablet">(.*?)</td>', block)
                if carrier_best_rating_reg:
                    Best = carrier_best_rating_reg.group(1).strip()
                elif carrier_best_rating_reg_1:
                    Best = carrier_best_rating_reg_1.group(1).strip()
                else:
                    Best = ""
                
                print("Rank   :",Rank)
                print("Name   :",Name)
                print("Team   :",Team)
                print("Rating :",Rating)
                print("Best   :",Best)
                print("*****"*100)
                
                temp = {"Rank":Rank,"Name":Name,"Team":Team,"Rating":Rating,"Best":Best}
                mydict.append(temp)
                    
        else:
            print("Else_here")
        
        self.Insert_excel(mydict)
        print("Success")
            
if __name__ == '__main__':              
    url='https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting'
    Exp = ICC_Ranking(url)
            

            
            
            